# Proyecto 1 - Etapa 1

In [19]:
import pandas as pd
import nltk

from sklearn.preprocessing import LabelEncoder
from sklearn.tree import DecisionTreeClassifier 
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report, precision_score, recall_score, f1_score, accuracy_score

from sklearn.metrics import ConfusionMatrixDisplay

from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import KFold 
import matplotlib.pyplot as plt
import seaborn as sns 
from sklearn import tree
import sklearn as sklearn
from joblib import dump, load

from nltk import word_tokenize, sent_tokenize
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer, WordNetLemmatizer

In [20]:
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('stopwords-es')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/paulinaarrazola/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/paulinaarrazola/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Error loading stopwords-es: Package 'stopwords-es' not
[nltk_data]     found in index
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/paulinaarrazola/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

# 1. Carga de datos

In [21]:
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", 20)
news=pd.read_csv("./fake_news.csv", sep=";")
data= news.copy()

In [22]:
data.shape

(57063, 5)

In [23]:
data.sample(5)

,ID,Label,Titulo,Descripcion,Fecha
42023,ID,1,La Fiscalía pide 18 años para La Manada y rech...,El Tribunal Supremo acoge este viernes la vist...,21/06/2019
48604,ID,1,Sánchez dice a Guaidó que unas elecciones son ...,El presidente ha mantenido una conversación te...,24/01/2019
40210,ID,0,El BNG de Feijóo cede ante la presión del ala ...,No hemos hablado en ningún momento que el abor...,14/02/2023
47050,ID,0,Gabilondo: 'Quiero de Cifuentes un relato vero...,El portavoz socialista en la Asamblea de Madri...,03/04/2018
51753,ID,1,Una treintena de candidaturas optan a dirigir ...,Las elecciones que tendrán lugar entre el 12 y...,07/06/2020


#### JUSTIFICAR LO QUE PODEMOS VER Y LA INFORMACION QUE NOS ESTAN DANDOE ESTAS VISUALIZACIONES

# 2. Entendimiento de datos

In [24]:
data.dtypes

ID             object
Label           int64
Titulo         object
Descripcion    object
Fecha          object
dtype: object

no se hace .describe porque no tiene sentido en este caso porque como todos los datos son de categoricos menos el id no tiene sentido 

In [28]:
data.isnull().sum()

ID              0
Label           0
Titulo         16
Descripcion     0
Fecha           0
dtype: int64

In [ ]:
#Saber cuantas de las noticias que tenemos son falsas. DESCRIBIR RESULTADOS
porcentajes = data['Label'].value_counts(normalize=True) * 100
print(porcentajes)

Label
1    58.107706
0    41.892294
Name: proportion, dtype: float64


# 3. Limpieza de datos 

Se eliminan los registros repetidos porque son pocos y el motor se puede confundir si encuentra títulos iguales con descripciones diferentes (explciar mejor por que se podria confundir el motor)

In [26]:
data.drop_duplicates()

,ID,Label,Titulo,Descripcion,Fecha
0,ID,1,'The Guardian' va con Sánchez: 'Europa necesit...,El diario británico publicó este pasado jueves...,02/06/2023
1,ID,0,REVELAN QUE EL GOBIERNO NEGOCIO LA LIBERACIÓN ...,REVELAN QUE EL GOBIERNO NEGOCIO LA LIBERACIÓN ...,01/10/2023
2,ID,1,El 'Ahora o nunca' de Joan Fuster sobre el est...,El valencianismo convoca en Castelló su fiesta...,25/04/2022
3,ID,1,"Iglesias alienta a Yolanda Díaz, ERC y EH Bild...","En política, igual que hay que negociar con lo...",03/01/2022
4,ID,0,Puigdemont: 'No sería ninguna tragedia una rep...,"En una entrevista en El Punt Avui, el líder de...",09/03/2018
...,...,...,...,...,...
57058,ID,1,El Defensor del Pueblo reclama a la Comunidad ...,El gobierno regional han indicado que la atenc...,08/06/2021
57059,ID,0,El EQUO plantea ceder la presidencia de la Com...,Si la higiene democrática nos lleva a esa exig...,08/09/2020
57060,ID,1,Alberto Garzón: 'Que los Borbones son unos lad...,El coordinador federal de IU asegura que la mo...,12/07/2018
57061,ID,1,Vox exige entrar en el Gobierno de Castilla y ...,Santiago Abascal: Vox tiene el derecho y el de...,13/02/2022


# 4. Modelos

### 4.1 Random Forest (Paulina Arrázola)